# Imports

In [1]:
import numpy as np
from functools import partial

In [2]:
from copy import deepcopy
from functools import partial
from pprint import pprint
import os.path as osp

In [3]:
import sacred
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from torch.backends import cudnn

In [4]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from typing import NamedTuple, Optional, List

In [5]:
from models.matcher import MatchERT
from models.ingredient import model_ingredient, get_model

In [6]:
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from sacred import Ingredient

In [7]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset
from utils.metrics import *

In [8]:
from utils import pickle_load
from sacred import Experiment
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.data.dataset import FeatureDataset
ex = sacred.Experiment('RRT Evaluation', ingredients=[data_ingredient, model_ingredient], interactive=True)

In [9]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [10]:
feature_name = 'r50_gldv1'
set_name = 'train'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [11]:
cpu = False
cudnn_flag = 'benchmark'
temp_dir = osp.join('logs', 'temp')
resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'
seed = 0

In [12]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [13]:
train_data_dir = '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt'

In [14]:
gnd = pickle_load(osp.join(train_data_dir, gnd_name))

In [15]:
len(gnd['imlist'])

42678

In [16]:
gnd['simlist'][0], len(gnd['simlist'][0])

(['512px-Apollo_Victoria_Theatre.jpg',
  '512px-Margaret_Hamilton_1966.jpg',
  '512px-Hairspray_the_Musical_2012_Aberystwyth_Arts_Centre_Summer_Season_production_poster.png',
  '512px-Beauty_and_Beast.jpg',
  '512px-Use_of_Wondering.jpg',
  '512px-The_Sound_of_Music_Christopher_Plummer_and_Julie_Andrews.jpg',
  '512px-Guys_and_Dolls_(1978).jpg',
  '512px-LauraBellBundyApr10.jpg',
  '512px-Chicago3_(MdB).jpg',
  '512px-Alice_Ghostley_Cousin_Alice_Mayberry_RFD_1970.JPG',
  '512px-Delrae_Knutson_as_Eliza_Doolittle_c1986.jpg',
  '512px-Glinda.png',
  '512px-Helen_Westley.jpg',
  '512px-Alexander_Mann_-_Soubrette.jpg',
  '512px-Mary_Poppins_Returns_Logo_Black.svg.png',
  '512px-Miss_Alice_May_H6724_(cropped).jpg',
  '512px-Jellia_Jamb.jpg',
  '512px-Laurie_Beechman_1984.jpg',
  '512px-Silvi_Vrait_1989.jpg',
  '512px-Dorothy_Loudon.jpg',
  '512px-JLT2017.jpg',
  '512px-Patty_Duke_1975.JPG',
  '512px-South_Pacific_scene.jpg',
  '512px-Stephanie_Kurtzuba_at_the_premiere_of_Bad_Education_(48810

In [17]:
import time
time.time()

1651847119.664981

In [18]:
gnd['gnd'][1].keys()

dict_keys(['easy', 'hard', 'junk', 'neg', 'provenance_entity', 'ir_order', 'img_rank_dict', 'rank_img_dict', 'r_easy', 'r_hard', 'r_junk', 'r_neg', 'r_ir_order', 'anchor_idx', 'g_easy', 'g_hard', 'g_junk', 'g_neg'])

In [61]:
new_selection = []

for i in range(len(gnd['simlist'])):
    array = np.array(gnd['simlist'][i])
    random_array = np.random.choice(array, 100)
    new_selection.append(random_array)

In [65]:
def format_selection(selection, order=True):
    
    def preserve_order(array):
        new_array = []
        for e in array:
            if e in new_array:
                continue
            else:
                new_array.append(e)
        return new_array
    
    if order:
        return [preserve_order(sub) for sub in selection]
    else:
        return [list(set(sub)) for sub in selection]

In [88]:
sizes = [len(gnd['simlist'][i]) for i in range(len(gnd['simlist']))]

In [89]:
nn_inds, _ = np.meshgrid(np.arange(100), np.arange(len(gnd['qimlist'])))
nn_inds.shape, nn_inds

((1190, 100),
 array([[ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        ...,
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99],
        [ 0,  1,  2, ..., 97, 98, 99]]))

In [112]:
for i in range(nn_inds.shape[0]):
    if max(gnd['gnd'][i]['r_ir_order']) >= sizes[i]:
        print(i)

In [113]:
counter = 0
for i in range(nn_inds.shape[0]):
    nn_inds[i,:sizes[i]] = gnd['gnd'][i]['r_ir_order']

In [115]:
gnd['gnd'][0]['r_ir_order']

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42]

In [120]:
nn_inds[1101]==np.arange(0,100,1)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [70]:
start_time1 = time.time()

format_selection(new_selection)

end_time1 = time.time()

start_time2 = time.time()

format_selection(new_selection, order=False)

end_time2 = time.time()

end_time1 - start_time1, end_time2 - start_time2

(0.12027907371520996, 0.09176778793334961)

In [53]:
start_time = time.time()

for i in range(len(gnd['simlist'])):
    for _ in range(100):
        array = np.array(gnd['simlist'][i])
        random_array = np.random.choice(array, 100)
        new_array = []
        for e in random_array:
            if not(e in new_array):
                new_array.append(e)
            else: continue
end_time = time.time()

end_time - start_time

25.956560373306274

In [51]:
new_array, array, end_time - start_time

(['512px-Lilian_Garcia_WrestleMania_Axxess_2015.jpg',
  '512px-Bon_Jovi_at_Madison_Square_Garden_(33868597862).jpg',
  '512px-US_Navy_070221-N-7883G-020_Singer_and_performer_Carol_Woods_sings_to_service_members_in_the_hangar_bay_aboard_USS_Kitty_Hawk_(CV_63)_prior_to_the_visit_by_Vice_President_Dick_Cheney.jpg',
  '512px-Barlowgirllive.JPG',
  '512px-Bellagio_outside.jpg',
  '512px-MaryJBligeJan10.jpg',
  '512px-August_Burns_Red.png',
  '512px-Usher_Cannes_2016_retusche.jpg',
  '512px-Etta_James.jpg',
  '512px-Sheryl_Crow.jpg',
  '512px-Queen_Latifah.jpg',
  '512px-RobEvanTSO2012.JPG',
  '512px-Tift_Merritt_-_Seattle.jpg',
  '512px-Paula-Cole_2009.jpg',
  '512px-Beyoncé_-_Sasha_Fierce_true_logo.png',
  '512px-IhaveadreamMarines.jpg',
  '512px-GretchenWilsonApr10.jpg',
  '512px-Gabriela_Montero_(2007).jpg',
  '512px-Joan_baez01.jpg',
  '512px-Michelle_Williams,_2015_(cropped).jpg',
  '512px-Ann_Marie_Calhoun_appearing_with_Ian_Anderson_Sao_Paolo_BRAZIL_April_2007.jpg',
  '512px-Joseph_L

In [17]:
gnd['gnd'][47]

{'easy': ['512px-Achille_Compagnoni_1955.jpg',
  '512px-K2_2006b.jpg',
  '512px-Lino_Lacedelli_1954.jpg'],
 'hard': [],
 'junk': ['512px-15-152-Pluto-NewHorizons-HighResolution-20150714-IFV.jpg',
  '512px-The_Malaysian_Federal_Honour_-_Panglima_Jasa_Negara_(PJN).jpg',
  '512px-Dolakha_Jiri.jpg',
  '512px-Kobe_eddiebauer2560.jpg',
  '512px-Douglas_Mawson_resting.jpg',
  '512px-Chris_bundled_up_on_an_expedition.jpg',
  '512px-Nepal_-_Sagamartha_Trek_-_240_-_Tengboche_Monastery.jpg',
  '512px-Cho_Oyu.jpg',
  "512px-Joske's_Thumb_from_near_Montfort_Boys_Town,_Fiji.jpg",
  '512px-Everest_kalapatthar_crop.jpg',
  '512px-Andrew_Scott_Waugh_1861.jpg',
  '512px-Hugh_Ruttledge_1936.jpg',
  '512px-G3C_spacesuit_Gemini_3.jpg',
  '512px-Carstensz-expeditie.jpg',
  '512px-210th_Rescue_Squadron_-_1.jpg',
  '512px-Tengboche-70-Lhotse_und_Ama_Dablam-2007-gje.jpg',
  '512px-Padma_Bhushan_Bachendri_Pal_(cropped).jpg',
  '512px-Lhotse.jpg',
  '512px-Annapurna_I.jpg',
  '512px-George_Everest_-_Maull_&_Poly

In [18]:
i = 47
for j in gnd['gnd'][i]['r_easy']:
    new_query = gnd['gnd'][i]['rank_img_dict'][j]
    print(new_query)

512px-Achille_Compagnoni_1955.jpg
512px-K2_2006b.jpg
512px-Lino_Lacedelli_1954.jpg


In [19]:
i = 47
for j in gnd['gnd'][i]['r_easy']:
    new_query = gnd['gnd'][i]['rank_img_dict'][j]
    index = gnd['gnd'][i]['img_rank_dict'][new_query]
    print(new_query)
    print(index)
    gnd['simlist'][i][index]


512px-Achille_Compagnoni_1955.jpg
5
512px-K2_2006b.jpg
17
512px-Lino_Lacedelli_1954.jpg
9


In [20]:
gnd['gnd'][i]['r_easy'], gnd['gnd'][i]['r_junk'], 

([5, 17, 9],
 [0,
  13,
  1,
  2,
  3,
  16,
  14,
  15,
  4,
  18,
  6,
  19,
  20,
  7,
  8,
  21,
  22,
  24,
  23,
  10,
  11,
  12])

In [21]:
not('512px-Achille_Compagnoni_1955.jpg' in gnd['simlist'][i])

True

In [22]:
len(gnd['simlist'][i])

22

In [23]:
[12, 8, 4, 15, 11, 23, 17]

[12, 8, 4, 15, 11, 23, 17]

In [24]:
gnd['gnd'][1151]['r_easy']

[0, 1]

In [25]:
for i in range(len(gnd['gnd'])):
    if len(gnd['gnd'][i]['r_junk']) == 0:
        print(i)

1151
